In [16]:
%matplotlib inline

In [1]:
import os
import io
import argparse
import importlib
import h5py
import numpy as np
import pandas as pd
import scipy.io as sio
import multiprocessing as mp
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

from PIL import Image

In [2]:
def rcparam():
    plt.rc('axes', linewidth=4.0, labelsize=18)
    # axes and axes labels properties
    plt.rc('xtick.major', size=10)  # length for x
    plt.rc('ytick.major', size=10)  # length for y
    plt.rc('lines', mew=5)
    plt.rc('lines', lw=4)  # line thickness
    plt.rc('ytick', labelsize=12)  # ytick label size
    plt.rc('xtick', labelsize=12)  # xtick label size
    plt.rc('xtick.major', pad=5)  # xtick padding distance
    plt.rc('ytick.major', pad=5)  # ytick padding distance


def plot_2d_heatmap(data, title=None, axis_labels=None, axis_tick_format='%.0f', force_matching_ticks=True, save_fig=False, fig_name='HeatMap.png', **kwargs):
    rcparam()
    font = {'fontsize': 20,
            'fontweight' : 'bold',
            'verticalalignment': 'baseline'}

    # Figure Creation
    fig, ax = plt.subplots()
    if 'invert_color' in kwargs:
        if kwargs['invert_color']:
            plt.set_cmap('viridis_r')
    if 'absolute_max' in kwargs:
        cax = ax.pcolormesh(data, vmin=0., vmax=kwargs['absolute_max'])
    else:
        cax = ax.pcolormesh(data)
    cbar = fig.colorbar(cax)

    # Adding labels
    if title:
        plt.title(title, fontdict=font)
    if axis_labels:
        plt.xlabel(axis_labels[0])
        plt.ylabel(axis_labels[1])
    if force_matching_ticks:
        xticks = ax.get_xticks()[0:-1]
        ax.set_yticks(xticks)

    ax.xaxis.set_major_formatter(FormatStrFormatter(axis_tick_format))
    ax.yaxis.set_major_formatter(FormatStrFormatter(axis_tick_format))
    
    #buf = io.BytesIO()
    if save_fig:
        plt.savefig(fig_name, transparent=True, bbox_inches='tight', pad_inches=0.1, format='png')
        #plt.savefig(buf, format='png')
    #plt.show()
    plt.close()
    #buf.seek(0)
    #return buf.read()
    

In [3]:
h5_sim_path = '../Simulation_Results/Trial_0001-py/simulation_data.h5'
h5_analysis_path = '../Simulation_Results/Trial_0001-py/Analysis/analysis_outputs.h5'

In [4]:
sim_h5_file = h5py.File(h5_sim_path, "r")
prob2d = np.array(sim_h5_file['Prob2D/set_00001'])
sim_h5_file.close()

In [6]:
plot_2d_heatmap(prob2d, title='Probability', save_fig=True)

img = Image.open("HeatMap.png")
data = np.asarray((img), dtype="uint8")

In [8]:
analysis_h5_file = h5py.File(h5_analysis_path, "w")

analysis_h5_file.create_dataset('image_test', data=data, dtype='uint8')

dset = analysis_h5_file.get("image_test")
dset.attrs['CLASS'] = np.string_('IMAGE')
dset.attrs['IMAGE_VERSION'] = np.string_('1.2')
arr = np.asarray([0, 255], dtype=np.uint8)
dset.attrs['IMAGE_MINMAXRANGE'] = list(arr)
dset.attrs['IMAGE_SUBCLASS'] = np.string_('IMAGE_TRUECOLOR')
dset.attrs['INTERLACE_MODE'] = np.string_('INTERLACE_PLANE')

analysis_h5_file.close()